In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, PULP_CBC_CMD
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Cereal grains and their products.csv"
df_cereal = pd.read_csv(path)
print(df_cereal.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Fats_ oils and fatty foods.csv"
df_nut = pd.read_csv(path)
print(df_cereal.head(1))


path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Fruits.csv"
df_fruit = pd.read_csv(path)
print(df_fruit.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Green leafy vegetables.csv"
df_leafy = pd.read_csv(path)
print(df_leafy.head(1))


path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Milk and milk products.csv"
df_milk = pd.read_csv(path)
print(df_milk.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Meat_ fish_ poultry and egg.csv"
df_meat = pd.read_csv(path)
print(df_meat.head(1))


path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Other vegetables_ roots and tubers.csv"
df_root = pd.read_csv(path)
print(df_root.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Pulses_ legumes and their products.csv"
df_pulse = pd.read_csv(path)
print(df_pulse.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Some indigenious foods.csv"
df_indi = pd.read_csv(path)
print(df_indi.head(1))

path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Sugar and similar products.csv"
df_sugar = pd.read_csv(path)
print(df_sugar.head(1))


path = "../yoges/Desktop/Deerwalk/dataset/datasets_diet/Wild edible plants.csv"
df_wild = pd.read_csv(path)
print(df_wild.head(1))


   ID common name        scientific name nepali name edible part, %  \
0   1       bajra  Pennisetum typhoideum       bajra             84   

  moisture, g protein, g fat, g minerals, g fiber, g carbohydrate, g  \
0        12.4       11.6    5.0         2.3      1.2            67.5   

  energy, kcal calcium, mg phosphorus, mg iron, mg carotene, mg or IU  \
0          361          42            296      5.0                132   

  thiamine, mg riboflavin, mg niacin, mg vitamin C, mg  
0         0.33           0.25        2.3             0  
   ID common name        scientific name nepali name edible part, %  \
0   1       bajra  Pennisetum typhoideum       bajra             84   

  moisture, g protein, g fat, g minerals, g fiber, g carbohydrate, g  \
0        12.4       11.6    5.0         2.3      1.2            67.5   

  energy, kcal calcium, mg phosphorus, mg iron, mg carotene, mg or IU  \
0          361          42            296      5.0                132   

  thiamine, mg r

In [3]:
# Concatenate datasets
df_original = pd.concat([
    df_cereal,
    df_nut,
    df_fruit,
    df_leafy,
    df_milk,
    df_meat,
    df_root,
    df_pulse,
    df_indi,
    df_sugar,
    df_wild
], ignore_index=True)

In [4]:
# Drop unnecessary columns
columns_to_drop = ['scientific name', 'nepali name', 'edible part, %', 'moisture, g', 'ID']
df_original = df_original.drop(columns=columns_to_drop, errors='ignore')

In [5]:
# Replace placeholders with NaN
object_columns = df_original.select_dtypes(include=['object']).columns
df_original[object_columns] = df_original[object_columns].replace(['--', 'NA', 'NaN', '', 'traces'], pd.NA)

In [6]:
# Define nutrient columns
nutrient_columns = [
    'protein, g', 'fat, g', 'fiber, g', 'carbohydrate, g', 'energy, kcal',
    'calcium, mg', 'phosphorus, mg', 'iron, mg', 'thiamine, mg',
    'riboflavin, mg', 'niacin, mg', 'vitamin C, mg'
]

In [7]:
# Convert nutrient columns to numeric
for col in nutrient_columns:
    if col in df_original.columns:
        df_original[col] = pd.to_numeric(df_original[col], errors='coerce').fillna(0)

In [8]:
# Handle 'carotene, mg or IU' column
if 'carotene, mg or IU' in df_original.columns:
    df_original['carotene, mg or IU'] = pd.to_numeric(df_original['carotene, mg or IU'], errors='coerce').fillna(0)

In [9]:
# Fill missing categorical columns
categorical_columns = df_original.select_dtypes(include=['object']).columns
df_original[categorical_columns] = df_original[categorical_columns].fillna('unknown')

In [10]:
# Standardize nutrient columns
scaler = StandardScaler()
df_original[nutrient_columns] = scaler.fit_transform(df_original[nutrient_columns])

In [11]:
# Define food datasets dictionary
food_datasets = {
    'df_cereal': df_cereal,
    'df_nut': df_nut,
    'df_fruit': df_fruit,
    'df_leafy': df_leafy,
    'df_milk': df_milk,
    'df_meat': df_meat,
    'df_root': df_root,
    'df_pulse': df_pulse,
    'df_indi': df_indi,
    'df_sugar': df_sugar,
    'df_wild': df_wild
}

In [12]:
def get_user_input():
    """Collect user input with validation."""
    user_data = {}
    
    try:
        user_data['age'] = int(input("Enter your age (1-120): "))
        if not 1 <= user_data['age'] <= 120:
            raise ValueError("Age must be between 1 and 120.")
        
        user_data['gender'] = input("Enter your gender (M/F): ").strip().lower()
        if user_data['gender'] not in ['m', 'f']:
            raise ValueError("Gender must be 'M' or 'F'.")
        
        user_data['height'] = float(input("Enter your height in cm (100-250): "))
        if not 100 <= user_data['height'] <= 250:
            raise ValueError("Height must be between 100 and 250 cm.")
        
        user_data['weight'] = float(input("Enter your weight in kg (30-200): "))
        if not 30 <= user_data['weight'] <= 200:
            raise ValueError("Weight must be between 30 and 200 kg.")
        
        user_data['activity_level'] = input(
            "Enter your activity level (sedentary, lightly active, moderately active, very active): "
        ).strip().lower()
        if user_data['activity_level'] not in ['sedentary', 'lightly active', 'moderately active', 'very active']:
            raise ValueError("Invalid activity level.")
        
        user_data['diet_type'] = input(
            "Enter your preferred diet type (vegan, vegetarian, omnivorous): "
        ).strip().lower()
        if user_data['diet_type'] not in ['vegan', 'vegetarian', 'omnivorous']:
            raise ValueError("Invalid diet type.")
        
        user_data['health_goal'] = input(
            "Enter your health goal (weight loss, gain, maintenance, cholesterol, diabetes): "
        ).strip().lower()
        if user_data['health_goal'] not in ['weight loss', 'gain', 'maintenance', 'cholesterol', 'diabetes']:
            raise ValueError("Invalid health goal.")
        
        # Structured food intake input
        print("\n--- Enter your food intake in the last 24 hours ---")
        food_intake = {}
        for meal in ['breakfast', 'lunch', 'dinner']:
            print(f"\nEnter {meal} items in format: food_name:amount_in_grams (semicolon-separated, or leave blank)")
            meal_input = input(f"{meal.title()} items: ").split(';')
            items = []
            for item in meal_input:
                item = item.strip()
                if not item:
                    continue
                try:
                    # Split on the last colon to handle commas in food names
                    parts = item.rsplit(':', 1)
                    if len(parts) != 2:
                        raise ValueError("Invalid format.")
                    name, grams = parts
                    grams = float(grams.strip())
                    if grams <= 0:
                        raise ValueError("Amount must be positive.")
                    # Normalize food name (e.g., milk_cow -> milk, cow)
                    name = name.strip().lower().replace('_', ' ').replace('  ', ' ')
                    if '(whole)' in name:
                        name = name.replace('(whole)', '(whole)')
                    items.append({'food': name, 'amount_g': grams})
                except ValueError as e:
                    print(f"Invalid format for item '{item}'. Skipped. Reason: {e}")
            food_intake[meal] = items
        user_data['food_intake'] = food_intake
    
    except ValueError as e:
        print(f"Error: {e}")
        return None
    
    return user_data

In [13]:
def calculate_bmi(weight, height_cm):
    """Calculate BMI."""
    height_m = height_cm / 100
    return round(weight / (height_m ** 2), 2)

In [14]:
def calculate_bmr(weight, height_cm, age, gender):
    """Calculate BMR using Mifflin-St Jeor equation."""
    if gender == 'm':
        return round(10 * weight + 6.25 * height_cm - 5 * age + 5, 2)
    return round(10 * weight + 6.25 * height_cm - 5 * age - 161, 2)

In [15]:
def calculate_tdee(bmr, activity_level):
    """Calculate TDEE based on activity level."""
    activity_factors = {
        'sedentary': 1.2,
        'lightly active': 1.375,
        'moderately active': 1.55,
        'very active': 1.725
    }
    multiplier = activity_factors.get(activity_level.lower(), 1.2)
    return round(bmr * multiplier, 2)

In [16]:
def calculate_health_metrics(user_data):
    """Calculate health metrics (BMI, BMR, TDEE)."""
    bmi = calculate_bmi(user_data['weight'], user_data['height'])
    bmr = calculate_bmr(user_data['weight'], user_data['height'], user_data['age'], user_data['gender'])
    tdee = calculate_tdee(bmr, user_data['activity_level'])
    return {"BMI": bmi, "BMR": bmr, "TDEE": tdee}

In [17]:
def find_food_nutrient(food_name, food_datasets):
    """Find nutrient data for a given food."""
    food_name = food_name.lower().strip()
    for df in food_datasets.values():
        # Case-insensitive match
        match = df[df['common name'].str.lower().str.strip() == food_name]
        if not match.empty:
            return match.iloc[0].to_dict()
    return None

In [18]:
def analyze_meal(meal_items, food_datasets, scaler):
    """Analyze nutrient content of a meal."""
    meal_nutrients = {
        "energy, kcal": 0,
        "protein, g": 0,
        "fat, g": 0,
        "carbohydrate, g": 0,
        "fiber, g": 0,
        "iron, mg": 0
    }
    
    for item in meal_items:
        food_name = item['food']
        amount = item['amount_g']
        nutrient_data = find_food_nutrient(food_name, food_datasets)
        
        if nutrient_data:
            for nutrient in meal_nutrients:
                value = nutrient_data.get(nutrient, 0)
                if pd.isna(value) or value in ['--', 'traces']:
                    value = 0
                try:
                    # Reverse standardization to get original values
                    if nutrient in nutrient_columns:
                        value = scaler.inverse_transform([[float(value)] + [0]*(len(nutrient_columns)-1)])[0][0]
                    meal_nutrients[nutrient] += (float(value) * amount / 100)
                except (ValueError, TypeError) as e:
                    print(f"Error processing nutrient '{nutrient}' for food '{food_name}': {e}. Using 0.")
                    meal_nutrients[nutrient] += 0
        else:
            print(f"⚠️ Food '{food_name}' not found in datasets.")
    
    return {k: round(v, 2) for k, v in meal_nutrients.items()}

In [19]:
def analyze_all_meals(user_data, food_datasets, scaler):
    """Analyze all meals and compute daily totals."""
    report = {}
    daily_total = {
        "energy, kcal": 0,
        "protein, g": 0,
        "fat, g": 0,
        "carbohydrate, g": 0,
        "fiber, g": 0,
        "iron, mg": 0
    }
    
    for meal in ['breakfast', 'lunch', 'dinner']:
        meal_items = user_data['food_intake'].get(meal, [])
        meal_nutrients = analyze_meal(meal_items, food_datasets, scaler)
        report[meal] = meal_nutrients
        for nutrient in daily_total:
            daily_total[nutrient] += meal_nutrients.get(nutrient, 0)
    
    report['total_day'] = {k: round(v, 2) for k, v in daily_total.items()}
    return report

In [20]:
def get_rda_targets(user, tdee):
    """Calculate RDA targets based on user profile and health goals."""
    weight = user['weight']
    goal = user['health_goal'].lower()
    
    rda = {}
    if goal == 'weight loss':
        rda['energy, kcal'] = tdee - 500
        rda['protein, g'] = 1.3 * weight
        fat_pct = 0.25
        carb_pct = 0.50
    elif goal == 'weight gain':
        rda['energy, kcal'] = tdee + 400
        rda['protein, g'] = 1.6 * weight
        fat_pct = 0.30
        carb_pct = 0.50
    elif goal == 'cholesterol':
        rda['energy, kcal'] = tdee
        rda['protein, g'] = 1.2 * weight
        fat_pct = 0.20
        carb_pct = 0.60
    elif goal == 'diabetes':
        rda['energy, kcal'] = tdee
        rda['protein, g'] = 1.3 * weight
        fat_pct = 0.25
        carb_pct = 0.45
    else:  # maintenance
        rda['energy, kcal'] = tdee
        rda['protein, g'] = 0.8 * weight
        fat_pct = 0.30
        carb_pct = 0.50
    
    rda['fat, g'] = (rda['energy, kcal'] * fat_pct) / 9
    rda['carbohydrate, g'] = (rda['energy, kcal'] * carb_pct) / 4
    rda['fiber, g'] = 25 if user['gender'] == 'f' else 38  # Standard RDA
    rda['iron, mg'] = 18 if user['gender'] == 'f' else 8  # Standard RDA
    
    return {k: round(v, 2) for k, v in rda.items()}

In [21]:
def calculate_nutrient_gaps(actual, required):
    """Calculate nutrient gaps between consumed and required amounts."""
    gaps = {}
    for nutrient in required:
        consumed = actual.get(nutrient, 0)
        required_val = required[nutrient]
        gap = required_val - consumed
        percent_deficit = (gap / required_val) * 100 if required_val > 0 else 0
        gaps[nutrient] = {
            "required": round(required_val, 2),
            "consumed": round(consumed, 2),
            "deficit": round(gap, 2),
            "deficit_%": round(percent_deficit, 1)
        }
    return gaps

In [22]:
def check_meal_diversity(user_input, datasets):
    """Check the diversity of food groups in meals."""
    diversity_report = {}
    for meal, items in user_input['food_intake'].items():
        groups_found = set()
        for item in items:
            for group, df in datasets.items():
                if item['food'].lower() in df['common name'].str.lower().values:
                    groups_found.add(group)
        diversity_report[meal] = {
            "food_groups": sorted(groups_found),
            "diversity_score": len(groups_found)
        }
    return diversity_report

In [23]:
def filter_foods(food_datasets, user_diet_type):
    """Filter foods based on diet type."""
    excluded_groups = {
        'vegan': ['df_milk', 'df_meat'],
        'vegetarian': ['df_meat'],
        'omnivorous': []
    }
    
    filtered_items = []
    for group_name, df in food_datasets.items():
        if group_name in excluded_groups[user_diet_type]:
            continue
        for _, row in df.iterrows():
            row_data = row.to_dict()
            row_data['source_group'] = group_name
            filtered_items.append(row_data)
    return filtered_items

In [24]:
def optimized_meal_recommendation(user, rda, allowed_foods, meal_type, scaler):
    """Generate optimized meal plan using Linear Programming."""
    prob = LpProblem(f"{meal_type}_Optimization", LpMinimize)
    
    # Variables: amount (grams) for each food (0 to 200g)
    amounts = {f['common name']: LpVariable(f['common name'], 0, 200) for f in allowed_foods}
    
    # Objective: Minimize squared deviation from RDA targets for the meal (1/3 of daily)
    target_nutrients = ['energy, kcal', 'protein, g', 'fat, g', 'carbohydrate, g', 'fiber, g', 'iron, mg']
    obj = 0
    nutrient_sums = {nutrient: lpSum([f[nutrient] * amounts[f['common name']] / 100 for f in allowed_foods]) for nutrient in target_nutrients}
    
    for nutrient in target_nutrients:
        target = rda[nutrient] / 3
        # Reverse standardization for original nutrient values
        actual = lpSum([scaler.inverse_transform([[f[nutrient]] + [0]*(len(nutrient_columns)-1)])[0][0] * amounts[f['common name']] / 100 for f in allowed_foods])
        obj += (actual - target) ** 2
    prob += obj
    
    # Constraints
    # 1. Energy within 90-110% of target
    prob += nutrient_sums['energy, kcal'] >= (rda['energy, kcal'] / 3) * 0.9
    prob += nutrient_sums['energy, kcal'] <= (rda['energy, kcal'] / 3) * 1.1
    
    # 2. Minimum and maximum foods (2-5 per meal)
    prob += lpSum([1 for f in allowed_foods if amounts[f['common name']] > 0]) >= 2
    prob += lpSum([1 for f in allowed_foods if amounts[f['common name']] > 0]) <= 5
    
    # 3. Ensure diversity: at least 2 different food groups
    food_groups = set(f['source_group'] for f in allowed_foods)
    for group in food_groups:
        prob += lpSum([amounts[f['common name']] for f in allowed_foods if f['source_group'] == group]) >= 0
    
    # Solve
    status = prob.solve(PULP_CBC_CMD(msg=0))
    if status != 1:
        print(f"Warning: {meal_type} optimization failed. Using fallback.")
        return []
    
    # Extract meal plan
    meal_plan = []
    for f in allowed_foods:
        amount = amounts[f['common name']].varValue
        if amount and amount > 1:  # Only include foods with >1g
            # Reverse standardization for nutrient values
            nutrients = {n: scaler.inverse_transform([[f[n]] + [0]*(len(nutrient_columns)-1)])[0][0] if n in nutrient_columns else f[n] for n in nutrient_columns}
            meal_plan.append({
                "food": f['common name'],
                "amount_g": round(amount, 2),
                "energy": round(nutrients['energy, kcal'] * amount / 100, 2),
                "protein": round(nutrients['protein, g'] * amount / 100, 2),
                "fat": round(nutrients['fat, g'] * amount / 100, 2),
                "carbs": round(nutrients['carbohydrate, g'] * amount / 100, 2),
                "fiber": round(nutrients['fiber, g'] * amount / 100, 2),
                "iron": round(nutrients['iron, mg'] * amount / 100, 2),
                "group": f['source_group']
            })
    
    return meal_plan

In [25]:
def score_meal(meal_items, rda_targets):
    """Score a meal based on nutrient coverage and balance."""
    total_energy = sum(item['energy'] for item in meal_items)
    total_protein = sum(item['protein'] for item in meal_items)
    total_fat = sum(item['fat'] for item in meal_items)
    total_carbs = sum(item['carbs'] for item in meal_items)
    total_fiber = sum(item['fiber'] for item in meal_items)
    total_iron = sum(item['iron'] for item in meal_items)
    total = total_protein * 4 + total_fat * 9 + total_carbs * 4
    
    # Nutrient coverage score (50 points)
    score_nutrient = 0
    for nutrient, actual, required in [
        ("energy, kcal", total_energy, rda_targets['energy, kcal'] / 3),
        ("protein, g", total_protein, rda_targets['protein, g'] / 3),
        ("fat, g", total_fat, rda_targets['fat, g'] / 3),
        ("carbohydrate, g", total_carbs, rda_targets['carbohydrate, g'] / 3),
        ("fiber, g", total_fiber, rda_targets['fiber, g'] / 3),
        ("iron, mg", total_iron, rda_targets['iron, mg'] / 3),
    ]:
        coverage = min(actual / required, 1.0) if required > 0 else 0
        score_nutrient += coverage * (50 / 6)  # 6 nutrients
    
    # Macronutrient balance score (30 points)
    score_macro = 0
    if total > 0:
        protein_pct = (total_protein * 4) / total
        fat_pct = (total_fat * 9) / total
        carb_pct = (total_carbs * 4) / total
        score_macro = (
            10 - abs(protein_pct - 0.2) * 50 +  # Ideal ~20%
            10 - abs(fat_pct - 0.3) * 33.3 +    # Ideal ~30%
            10 - abs(carb_pct - 0.5) * 20       # Ideal ~50%
        )
        score_macro = max(min(score_macro, 30), 0)
    
    # Diversity score (20 points)
    food_groups = set(item['group'] for item in meal_items)
    diversity_score = min(len(food_groups) / 5, 1.0) * 20
    
    return round(score_nutrient + score_macro + diversity_score, 2)

In [26]:
def generate_final_report(user, health_metrics, rda_targets, nutrient_gaps, structured_meals, meal_scores, diversity_report):
    """Generate a personalized diet report."""
    report = []
    report.append("\n FINAL PERSONALIZED DIET REPORT")
    report.append("=" * 50)
    
    report.append("\n Health Metrics:")
    for k, v in health_metrics.items():
        report.append(f"- {k}: {v}")
    
    report.append("\n Nutrient Intake vs Requirement:")
    for nutrient, val in nutrient_gaps.items():
        status = "Deficit" if val['deficit'] > 0 else "Excess" if val['deficit'] < 0 else "Met"
        report.append(f"- {nutrient.replace(', ', ' ').title()}: {val['consumed']} / {val['required']} ({status}: {abs(val['deficit'])} | {val['deficit_%']:.1f}%)")
    
    report.append("\n Recommended Meal Plan:")
    for meal, items in structured_meals.items():
        report.append(f"\n{meal.title()} (Score: {meal_scores[meal]}/100):")
        for food in items:
            report.append(
                f"  • {food['food'].title()} ({food['amount_g']}g): "
                f"{food['energy']:.1f} kcal, {food['protein']:.1f}g protein, "
                f"{food['carbs']:.1f}g carbs, {food['fat']:.1f}g fat, "
                f"{food['fiber']:.1f}g fiber, {food['iron']:.1f}mg iron"
            )
    
    report.append("\n Meal Diversity:")
    for meal, d in diversity_report.items():
        report.append(f"- {meal.title()}: {d['diversity_score']} food groups ({', '.join(d['food_groups'])})")
    
    report.append("\n Tips for Improvement:")
    for nutrient, gap in nutrient_gaps.items():
        if gap["deficit_%"] > 20:
            if "iron" in nutrient.lower():
                report.append("- Increase iron intake with leafy greens (e.g., agathi) or legumes (e.g., bengal gram).")
            elif "protein" in nutrient.lower():
                report.append("- Boost protein with pulses, tofu, or dairy (if not vegan).")
            elif "carbohydrate" in nutrient.lower():
                report.append("- Add whole grains like bajra or rice for healthy carbs.")
            elif "fat" in nutrient.lower():
                report.append("- Include healthy fats from nuts or oils like mustard oil.")
            elif "energy" in nutrient.lower():
                report.append("- Increase calorie intake with nutrient-dense foods like pulses or grains.")
            elif "fiber" in nutrient.lower():
                report.append("- Boost fiber with vegetables, fruits, or whole grains.")
    
    report.append("=" * 50)
    
    return "\n".join(report)

In [58]:
# def main():
#     """Main function to run the diet analysis and recommendation system."""
#     try:
#         # Get user input
#         user_data = get_user_input()
#         if not user_data:
#             print("Failed to collect user input. Exiting.")
#             return
        
#         # Calculate health metrics
#         health_metrics = calculate_health_metrics(user_data)
        
#         # Analyze meals
#         meal_analysis = analyze_all_meals(user_data, food_datasets, scaler)
        
#         # Calculate RDA targets
#         rda_targets = get_rda_targets(user_data, health_metrics['TDEE'])
        
#         # Calculate nutrient gaps
#         nutrient_gaps = calculate_nutrient_gaps(meal_analysis['total_day'], rda_targets)
        
#         # Check meal diversity
#         diversity_report = check_meal_diversity(user_data, food_datasets)
        
#         # Filter foods based on diet
#         allowed_foods = filter_foods(food_datasets, user_data['diet_type'])
#         if not allowed_foods:
#             print("No foods available after filtering. Check diet restrictions.")
#             return
        
#         # Generate optimized meal recommendations
#         structured_meals = {
#             meal: optimized_meal_recommendation(user_data, rda_targets, allowed_foods, meal, scaler)
#             for meal in ['breakfast', 'lunch', 'dinner']
#         }
        
#         # Score meals
#         meal_scores = {meal: score_meal(items, rda_targets) for meal, items in structured_meals.items()}
        
#         # Generate and print final report
#         report = generate_final_report(user_data, health_metrics, rda_targets, nutrient_gaps, structured_meals, meal_scores, diversity_report)
#         print(report)
        
#         # Save report to file
#         with open("diet_report.txt", "w") as f:
#             f.write(report)
#         print("\nReport saved to 'diet_report.txt'")
    
#     except Exception as e:
#         print(f"An error occurred: {e}")

def main():
    """Main function to run the diet analysis and recommendation system."""
    try:
        # Get user input
        user_data = get_user_input()
        if not user_data:
            print("Failed to collect user input. Exiting.")
            return
        
        # Calculate health metrics
        health_metrics = calculate_health_metrics(user_data)
        
        # Analyze meals
        meal_analysis = analyze_all_meals(user_data, food_datasets, scaler)
        
        # Calculate RDA targets
        rda_targets = get_rda_targets(user_data, health_metrics['TDEE'])
        
        # Calculate nutrient gaps
        nutrient_gaps = calculate_nutrient_gaps(meal_analysis['total_day'], rda_targets)
        
        # Check meal diversity
        diversity_report = check_meal_diversity(user_data, food_datasets)
        
        # Filter foods based on diet
        allowed_foods = filter_foods(food_datasets, user_data['diet_type'])
        if not allowed_foods:
            print("No foods available after filtering. Check diet restrictions.")
            return
        
        # Generate optimized meal recommendations
        structured_meals = {
            meal: optimized_meal_recommendation(user_data, rda_targets, allowed_foods, meal, scaler)
            for meal in ['breakfast', 'lunch', 'dinner']
        }
        
        # Score meals
        meal_scores = {meal: score_meal(items, rda_targets) for meal, items in structured_meals.items()}
        
        # Generate and print final report
        report = generate_final_report(user_data, health_metrics, rda_targets, nutrient_gaps, structured_meals, meal_scores, diversity_report)
        print(report)
        
        # Save report to file
        with open("diet_report.txt", "w") as f:
            f.write(report)
        print("\nReport saved to 'diet_report.txt'")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Enter your age (1-120):  30
Enter your gender (M/F):  M
Enter your height in cm (100-250):  170
Enter your weight in kg (30-200):  70
Enter your activity level (sedentary, lightly active, moderately active, very active):  moderately active
Enter your preferred diet type (vegan, vegetarian, omnivorous):  omnivorous
Enter your health goal (weight loss, gain, maintenance, cholesterol, diabetes):  maintenance



--- Enter your food intake in the last 24 hours ---

Enter breakfast items in format: food_name:amount_in_grams (semicolon-separated, or leave blank)


Breakfast items:  bajra:100; curd:50



Enter lunch items in format: food_name:amount_in_grams (semicolon-separated, or leave blank)


Lunch items:  bajra:100; curd:50



Enter dinner items in format: food_name:amount_in_grams (semicolon-separated, or leave blank)


Dinner items:  bajra:100; curd:50


An error occurred: expected string or bytes-like object, got 'float'


Traceback (most recent call last):
  File "C:\Users\yoges\AppData\Local\Temp\ipykernel_23572\1223865245.py", line 84, in main
    meal: optimized_meal_recommendation(user_data, rda_targets, allowed_foods, meal, scaler)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yoges\AppData\Local\Temp\ipykernel_23572\1085042985.py", line 6, in optimized_meal_recommendation
    amounts = {f['common name']: LpVariable(f['common name'], 0, 200) for f in allowed_foods}
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yoges\anaconda3\Lib\site-packages\pulp\pulp.py", line 263, in __init__
    LpElement.__init__(self, name)
  File "C:\Users\yoges\anaconda3\Lib\site-packages\pulp\pulp.py", line 179, in __init__
    self.name = name
    ^^^^^^^^^
  File "C:\Users\yoges\anaconda3\Lib\site-packages\pulp\pulp.py", line 163, in setName
    if self.expression.match(name):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeE

In [28]:
print(food_datasets['df_milk']['common name'].tolist())

['milk_cow', 'milk, buffalo', 'milk, goat', 'milk, human', 'curd', 'buttermilk', 'chhana, buffalo milk', 'cheese', 'khoa, whole buffalo milk', 'skimmed milk powder, cow milk', 'whole milk powder, cow milk']
